In [53]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import statsmodels.api as sm
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

On October 23th, 2013, Lehman Brothers Holdings Inc. and Lehman Brothers Special
Financing Inc. filed suit against Giants Stadium LLC in the United States Bankruptcy
Court for the Southern District of New York. The issue at stake is the termination
value of two interest rate swaps as of the date of Lehman’s bankruptcy on September
15th, 2008. Lehman claims that the value of the swaps was approximately $94,000,000
in their favor. Giants Stadium LLC argues that Lehman actually owes them nearly
$585,000,000 instead. The objective of this case is to explore the use of swaps, under-
stand better the risks of swaps and their legal environment, and to provide accurate
market-based valuations for the swaps.
In this case, you may assume that the background facts presented in paragraphs 25
through 59 of the attached legal complaint filed by Lehman Brothers are relevant. For
example, the relevant notional amount of the swaps is $408,325,000. The swaps have
the same maturity date as the underlying auction rate debt which is April 1, 2047.
The fixed rate of the swaps is 6.1885 percent.

In [54]:
notional = 408325000
fixed_rate = 0.061885
maturity_date = '2047-04-01'
issue_date = '2007-08-16'
data = {
    '2008-09-15': [2.50, 2.82, 3.00, 2.62, 2.93, 3.29, 3.54, 3.69, 3.98, 4.24, 4.61],
    '2008-09-18': [3.19, 3.20, 3.29, 2.99, 2.95, 3.25, 3.46, 3.61, 3.88, 4.13, 4.47],
}

# Create DataFrame and set index
libor_df = pd.DataFrame(data, index=['1-M LIBOR', '3-M LIBOR', '6-M LIBOR', '1-Y Swap Par', '2-Y Swap Par',
                                '3-Y Swap Par', '4-Y Swap Par', '5-Y Swap Par', '7-Y Swap Par',
                                '10-Y Swap Par', '30-Y Swap Par'])

# Display the DataFrame
print(libor_df)


               2008-09-15  2008-09-18
1-M LIBOR            2.50        3.19
3-M LIBOR            2.82        3.20
6-M LIBOR            3.00        3.29
1-Y Swap Par         2.62        2.99
2-Y Swap Par         2.93        2.95
3-Y Swap Par         3.29        3.25
4-Y Swap Par         3.54        3.46
5-Y Swap Par         3.69        3.61
7-Y Swap Par         3.98        3.88
10-Y Swap Par        4.24        4.13
30-Y Swap Par        4.61        4.47


1. Provide a concise explanation of how the plaintiff and defendant in this lawsuit are
each choosing to value the swaps.





In concise terms, the master contract for the swap contained 2 applicable methods for valuing the payable upon early termination:
-  The first stated that should Lehman still operate as the Broker-Dealer of the underlying ARS bonds at time of termination, the payable would amount to $[(LIBOR1M_t+0.003185) - 0.06185]*N$ where N is the notional swap amount and $ARS_t$ the current rate payed on the stadium's bonds. This method implies a payable of $94 Million by Giants Stadium to Lehman <br>
- The second method relies on the valuation clause in the case of a Lehman Bankruptcy event. In this case, it is stipulated that both parties shall agree upon 2 Designated Dealers for quotation and set the contractually agreed upon Market Quotation to be the lowest of these two values. In this case, the payables are deemed to amount to $301 Million in Giants Stadium's favor

2. Provide a brief critique of both of the valuation theories.


The critique of the Giants' valuation theory hinges on the enforceability of the Market Quotation clause. First, it argues that the clause is unenforceable in bankruptcy proceedings if it detrimentally affects the rights of the defaulting party. Furthermore, it claims that additional clauses acquired as compensation for bearing the risk of ARS instead of LIBOR on its books stipulate that upon an early termination requiring such market valuation, Lehman would be afforded the necessary time to solicit quotations and full prerogatives over the presentation of economic facts to designatd dealers in order to prevent undue impingement of the contract's values due to momentary market instability. Finally, it claims that invocation of termination clauses by Giants Stadium which leave it in default of its minimum hedging requirements must be expressly approved by Lehman to be enacted. <br>
Additionally, prior to Lehman's defaults, much of the bonds went unsold and thus were held by Lehman, in which case the floating rate was stipulated to amount to 90% of LIBOR, accordingly this left them roughly $60 Million in-the-money in a September '08 valuation statement which Giants Stadium did not contest.

Conversely, the issue with Lehmann's position is that under the conventional ISDA Master Agreement and commonly accepted practices around such transactions, the non-defaulting party is afforded the right to solicit settlement on market terms through the solicitation of such quotes. This is done in order that the contract be allowed to persist at its projected economic value should this be possible. Thus, although Lehman claims to have obtained a number of protective clauses as direct compensation for bearing the added risk of the ARS rate on its books, Giants Stadium can expect that its long-term hedge can be valued using a methodology which incorporates the long-term value it purchased the contract for. Lehman's position thus fails to reflect the economic value of the contract and is legally exposed should it be deemed to be seeking market valuation in bad faith (and/or to be interpreting the default clause in bad faith vis-a-vis the enforcement exemption it is arguing). 

3. Describe briefly the difference between traditional floating rate debt and auction
rate floating rate debt.
When answering questions 4-6, you will have to make simplifying assumptions about
the nature of the discount curve after 30 years. Please list your assumptions. Moreover,
in these questions, you may assume that the floating-rate experienced a reset on the
date when the swaps are being valued (i.e., the next floating-rate payment will occur
exactly one month into the future). Also assume that the last fixed payment will take
place in March 2047.


Traditional floating rate debt is usually pinned on a reference rate, agreed upon at time $t_0$ such as LIBOR (in this discussion around '08) or SOFR nowadays. The interest payment on such debt is thus made to mirror current market conditions, thus granting the bearer of the bond protection against duration risk. The canonical example of this type of debt are TIPSs, floating notes issued by the fed to investors.

Auction rate securities, while also providing a floating coupon payment determine the rate on the basis of an auction whereby investors bid down the interest rate which they are willing to accept for bearing a security. This mechanism is intended to provide a similar reflection of market conditions based on a live matching of supply and demand.

4. The auction rate securities market essentially collapsed in February 2008, with a
large number of auctions failing. The deterioration of the market caused Lehman to
purchase 100 percent of the bonds referenced by the swaps. As the bonds were ”held”,
as opposed to auctioned, the floating rate on the swaps was calculated using the All
Hold rate, which is 90 percent of the one-month LIBOR rate. Value the swaps as of
September 15, 2008 and September 18, 2008 using the All Hold rate.

Utilizing a simplified approach where swap is based on values of the fixed - floating payment. Not fixed - floating leg. 

In [56]:
# In this example I will calculate swap value based on one payment for simplicity.

# Calculate hold rate based on libor
hold_rate_sept_15 = .025 * .9
hold_rate_sept_18 = .0319 * .9

# Using LIBOR + .003185 as stated in paragraph 34
value_sept_15 = (fixed_rate - hold_rate_sept_15) * notional
value_sept_18 = (fixed_rate - hold_rate_sept_18) * notional

print('Swap value September 15, 2008: ', value_sept_15)
print('Swap value September 18, 2008: ', value_sept_18)

Swap value September 15, 2008:  16081880.125000002
Swap value September 18, 2008:  13546181.875000002


5. Giants Stadium terminated the swaps on September 18, 2008, 3 days after LBHI
filed for Chapter 11. Value the swaps on that date using the assumption that Lehman
would pay one-month Libor plus 31.85 basis points on the floating leg of the swap.

In [57]:
value_sept_18_plus_libor = (fixed_rate - (hold_rate_sept_18 + .003185)) * notional
print('Swap value September 18 including 31.85 basis points: ', value_sept_18_plus_libor)

Swap value September 18 including 31.85 basis points:  12245666.750000002


6. Finally, value the swaps on September 18, 2008 using the valuation method advocated by Giants Stadium. Specifically, the last auction rate on the Goldman Sachs
Bonds was 10.17 percent. Giants Stadium is arguing that the swaps should be valued
as if the floating rate paid by Lehman over the rest of the life of the swaps would be
fixed at 10.17 percent.

In [58]:
value_Goldman = (.1017 - .0447) * notional
print(value_Goldman)

23274525.0


7. Given the market events around the week of the Lehman bankruptcy, what other
factors might be relevant to consider in valuing the swaps?

It is also important to factor in market conditions overall. For instance, when Lehman went bankrupt financial crisis just began, meaning there was less liquidity in general making in harder to unwind or hedge.